In [5]:
%pip install split-folders
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/922.2 kB ? eta -:--:--
   --------------------------------------- 922.2/922.2 kB 10.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ------------------ --------------------- 3.7/8.1 MB 18.2 MB/s eta 0:00:01
   -------------------------------- ------- 6.6/8.1 MB 15.5 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 13.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   -- ------------------------------------- 2.6/39.5 MB 12.5 MB/s eta 0:00:03
   ---- ----------------------------------- 4.7/39.5 MB 11.4 MB/s eta 0:00:04
   ------- -------------------------------- 7.1/39.5 MB 11.5 MB/s eta 0:00:03
   -------- ------------------------------- 8.7/3

In [3]:
import splitfolders

input_folder = "./input-data"
output_folder = "./dataset"

splitfolders.ratio(input_folder,output = output_folder, seed=42, ratio=(0.8,0.1,0.1),group_prefix=None)

Copying files: 4408 files [00:10, 440.25 files/s]


In [14]:
from ultralytics import YOLO

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11n.pt")

# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(data="config.yaml", project="./output",device='gpu',epochs=100, batch=12, imgsz=512,workers=8, optimizer='SGD',lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=2.0 )

# Evaluate the model's performance on the validation set
results = model.val()

# Export the model to ONNX format
success = model.export(format="onnx")

Ultralytics 8.3.85  Python-3.11.11 torch-2.6.0+cpu 


ValueError: Invalid CUDA 'device=gpu' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: gpu
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


False
0
